<u>RNN(=再帰型ニューラルネットワーク)</u>   
ディープラーニングの構造．過去の情報を利用して現在，及び将来の入力に対するネットワークの性能を向上させる．    
<u>one-hot表現</u>    
単語とone-hotベクトルが1:1対応している．    
<u>ソフトマックス関数</u>    
入力されたベクトルの各成分を0.0~1.0の範囲の確率値に変換する関数．   
ソフトマックス関数によって出力されるベクトルの各成分の和は1.0(=100%)となる．

In [30]:
import torch
import torch.nn  as nn
from torch.utils.data import Dataset
import pandas as pd
from importnb import imports
with imports ("ipynb"):
    import  knock80 as k80

In [31]:
class RNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, emb_size, pad_idx, output_size):
        super().__init__()
        self.hid_size = hidden_size# 隠れ状態の特徴量ベクトルの数
        self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=pad_idx)
        """
        nn.Enbedding
        処理しやすい数値ベクトル(=埋め込みベクトル)に変換してあげる
        入力: {'this': 0, 'is': 1, 'a': 2, 'sentence': 3}
                だとすると[0, 1, 2, 3]みたいな単語IDの並び
        padding_idx: ベクトルを計算したくない部分のインデックス
        出力: [[単語ID 0 に対する埋め込みベクトル],
                [単語ID 1 に対する埋め込みベクトル],
                [単語ID 2 に対する埋め込みベクトル],
                [単語ID 3 に対する埋め込みベクトル]]
        """
        self.rnn = nn.RNN(emb_size, hidden_size, 
                          nonlinearity="tanh", batch_first=True)
        """
        nonlinearity: 使用する非線形の関数
        tanh: ハイパボリックタンジェント．あらゆる入力値を-1.0~1.0の範囲に変換する．
        batch_first: Trueの場合，入力や出力の際にbatchが先頭に来る．
        """
        self.fc = nn.Linear(hidden_size, output_size)# 線形変換
    
    def forward(self, x):
        self.batch_size = x.size()[0]# size():全要素数
        hidden = torch.zeros(1, self.batch_size, self.hid_size)#スカラー変換
        emb = self.emb(x)#埋め込みベクトルへの変換
        out, hidden = self.rnn(emb, hidden)#rnnを通して計算
        out = self.fc(out[:, -1, :])#最後の層を出力
        return out

In [32]:
class NewsDataset(Dataset):
    def __init__(self, x, y, tokenizer, word_dict):
        self.x = x
        self.y = y
        self.tokenizer = tokenizer#文字列を数値変換する
        self.word_dict = word_dict
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        text = self.x[idx]
        inputs = self.tokenizer(self.word_dict, text)

        #torch.tensor: tensor型に変換する
        return {
            "inputs": torch.tensor(inputs, dtype=torch.int64),
            "labels": torch.tensor(self.y[idx], dtype=torch.int64)
        }

In [33]:
train = pd.read_csv("../chapter06/train.txt", sep="\t")
valid = pd.read_csv("../chapter06/valid.txt", sep="\t")
test = pd.read_csv("../chapter06/test.txt", sep="\t")

category = {"b":0, "t":1, "e":2, "m":3}
y_train = torch.tensor(train["CATEGORY"].map(lambda x: category[x]).values)
y_valid = torch.tensor(valid["CATEGORY"].map(lambda x: category[x]).values)
y_test = torch.tensor(test["CATEGORY"].map(lambda x: category[x]).values)

In [34]:
dataset_train = NewsDataset(train["TITLE"], y_train, k80.find_id_by_word, k80.train_id)
dataset_valid = NewsDataset(valid["TITLE"], y_valid, k80.find_id_by_word, k80.train_id)
dataset_test = NewsDataset(test["TITLE"], y_test, k80.find_id_by_word, k80.train_id)

In [35]:
print(f"len(Dataset): {len(dataset_train)}")
print("Dataset[index]:")
for var in dataset_train[1]:
    print(f"{var}: {dataset_train[1][var]}")

VOCAB_SIZE = len(set(k80.train_id.values())) + 1
EMB_SIZE = 300
PADDING_IDX = len(set(k80.train_id.values()))
OUTPUT_SIZE = 4
HIDDEN_SIZE = 50

model = RNN(HIDDEN_SIZE, VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE)
for i in range(10):
    X = dataset_train[i]["inputs"]
    print(torch.softmax(model(X.unsqueeze(0)), dim=-1))
    #unsqueeze: 指定した位置にサイズ1の次元を挿入

len(Dataset): 10672
Dataset[index]:
inputs: tensor([2037,   16, 2022, 2501, 3538,  166,    5,   64,    0])
labels: 0
tensor([[0.1702, 0.0962, 0.2774, 0.4563]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1465, 0.0973, 0.2686, 0.4876]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2552, 0.3165, 0.2917, 0.1366]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1755, 0.0849, 0.2442, 0.4954]], grad_fn=<SoftmaxBackward0>)
tensor([[0.5121, 0.1467, 0.1631, 0.1781]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1383, 0.0889, 0.2930, 0.4798]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1622, 0.0996, 0.2630, 0.4752]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2551, 0.2008, 0.2316, 0.3126]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0951, 0.2406, 0.5217, 0.1425]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2083, 0.1032, 0.2392, 0.4493]], grad_fn=<SoftmaxBackward0>)


/var/folders/1p/m23jt2g15kz8d77xv0qpzbt40000gn/T/ipykernel_8652/679476676.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(self.y[idx], dtype=torch.int64)
